In [1]:
import numpy as np
import pandas as pd
import collections
import ast

In [4]:
pwd

'/home/likegiver/Documents/zhijie/11711_hw2'

In [5]:
all_df = pd.read_csv('gemma3_questions_filtered_tmp_with_2cat_concise.csv', lineterminator='\n')

In [6]:
annotated_df = pd.read_csv("iaa_answers.csv")

In [7]:
# F1 measures the word overlap between the labeled and the predicted answer
# tp: number of tokens* that are shared between the correct answer and the prediction.
# fp: number of tokens that are in the prediction but not in the correct answer.
# fn: number of tokens that are in the correct answer but not in the prediction.
def get_f1(annotated_answer, gen_answer):
    annotated_tokens = annotated_answer.split()
    gen_tokens = gen_answer.split()
    common = collections.Counter(annotated_tokens) & collections.Counter(gen_tokens)
    num_same = sum(common.values())
    
    if len(annotated_tokens) == 0 or len(gen_tokens) == 0:
        return int(annotated_tokens == gen_tokens)
    
    if num_same == 0:
        return 0
    
    precision = 1.0 * num_same / len(gen_tokens)
    recall = 1.0 * num_same / len(annotated_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    
    return f1

In [11]:
def get_annotator_f1(annotated_df, generated_df):
    f1 = 0
    for index, row in annotated_df.iterrows():
        question, annotated_answer = row[0], row[1]
        gen_answer = generated_df[generated_df["gemma3:12b_question"] == question]["gemma3:12b_answer_concise"].iloc[0]
        f1 += get_f1(annotated_answer, gen_answer)
    
    return f1 / annotated_df.shape[0]

In [12]:
annotator1_f1 = get_annotator_f1(annotated_df[["gemma3:12b_question", "annotated_answer1"]], all_df)
annotator2_f1 = get_annotator_f1(annotated_df[["gemma3:12b_question", "annotated_answer2"]], all_df)
overall_f1 = (annotator1_f1 + annotator2_f1) / 2

/tmp/ipykernel_1017332/164662950.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  question, annotated_answer = row[0], row[1]
/tmp/ipykernel_1017332/164662950.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  question, annotated_answer = row[0], row[1]


In [13]:
print(overall_f1)

0.21056926035525106
